In [1]:
import torch
import torch.nn as nn
import numpy as np
import nibabel as nib

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("awsaf49/brats20-dataset-training-validation")

print("Path to dataset files:", path)

/Users/enesdemir/Desktop/BrainTumorModel/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: /Users/enesdemir/.cache/kagglehub/datasets/awsaf49/brats20-dataset-training-validation/versions/1


In [10]:
import os 
from torch.utils.data import Dataset, DataLoader
from torchio import Subject, ScalarImage, LabelMap

class BraTSDataset(Dataset):
    def __init__(self, folders, transform=None):
        self.transform = transform
        self.folders = folders
    
    def __len__(self):
        return len(self.folders)
    
    def __getitem__(self, idx):
            flair_path = os.path.join(self.folders[idx], f"{os.path.basename(self.folders[idx])}_flair.nii")
            t1_path = os.path.join(self.folders[idx], f"{os.path.basename(self.folders[idx])}_t1.nii")
            t1ce_path = os.path.join(self.folders[idx], f"{os.path.basename(self.folders[idx])}_t1ce.nii")
            t2_path = os.path.join(self.folders[idx], f"{os.path.basename(self.folders[idx])}_t2.nii")
            seg_path = os.path.join(self.folders[idx], f"{os.path.basename(self.folders[idx])}_seg.nii")

            subject = Subject(
                flair=ScalarImage(flair_path),  
                t1ce=ScalarImage(t1ce_path),
                t2=ScalarImage(t2_path),
                seg=LabelMap(seg_path), 
                t1=ScalarImage(t1_path)
            )

            if self.transform:
                transformed_subject = self.transform(subject)
                input_tensor = torch.stack([
                    transformed_subject.flair.data,
                    transformed_subject.t1ce.data,
                    transformed_subject.t2.data,
                    transformed_subject.t1.data
                ], dim=0)  # Şekil: [4, 1, 128, 128, 128]
                label = transformed_subject.seg.data  # Şekil: [1, 128, 128, 128]
                
                # Etiketleri one-hot encoding ile dönüştür
                label = label.squeeze(0)  # Şekil: [128, 128, 128]
                # BraTS etiketlerini yeniden eşle (0, 1, 2, 4 -> 0, 1, 2, 3)
                label_mapped = torch.zeros_like(label)
                label_mapped[label == 1] = 1  # Nekroz
                label_mapped[label == 2] = 2  # Ödem
                label_mapped[label == 4] = 3  # Güçlendirici tümör
                
                # One-hot encoding
                label_one_hot = torch.zeros(4, 128, 128, 128)  # 4 sınıf için
                for c in range(4):
                    label_one_hot[c] = (label_mapped == c).float()
                label = label_one_hot  # Şekil: [4, 128, 128, 128]
            else:
                flair = torch.from_numpy(nib.load(flair_path).get_fdata()).float()
                t1ce = torch.from_numpy(nib.load(t1ce_path).get_fdata()).float()
                t2 = torch.from_numpy(nib.load(t2_path).get_fdata()).float()
                t1 = torch.from_numpy(nib.load(t1_path).get_fdata()).float()
                input_tensor = torch.stack([flair, t1ce, t2, t1], dim=0)
                label = torch.from_numpy(nib.load(seg_path).get_fdata()).float()
                
                # Etiketleri one-hot encoding ile dönüştür
                label_mapped = torch.zeros_like(label)
                label_mapped[label == 1] = 1
                label_mapped[label == 2] = 2
                label_mapped[label == 4] = 3
                
                label_one_hot = torch.zeros(4, 128, 128, 128)
                for c in range(4):
                    label_one_hot[c] = (label_mapped == c).float()
                label = label_one_hot

            return input_tensor, label

In [11]:
from sklearn.model_selection import train_test_split

path = '/Users/enesdemir/.cache/kagglehub/datasets/awsaf49/brats20-dataset-training-validation/versions/1'
real_path = os.path.join(path, 'BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData')

all_folders = [os.path.join(real_path,f) for f in os.listdir(real_path) if not f.endswith('.csv')]

train_folders, val_folders = train_test_split(all_folders, test_size=0.2, random_state=42)

Aşagıdaki data augmentiona araştırıcam

In [12]:
import torchio as tio

transform = tio.Compose([
    tio.ZNormalization(masking_method=tio.ZNormalization.mean),  # Normalizasyon
    tio.CropOrPad((128, 128, 128)),  # Kırpma
    tio.RandomAffine(degrees=15, scales=(0.9, 1.1)),  # Augmentation
    tio.RandomNoise(std=0.1),  # Gürültü ekleme
])

In [13]:
training_set = BraTSDataset(train_folders,transform=transform)
training_loader = DataLoader(training_set,
                             batch_size=4,
                             shuffle=True,
                             num_workers=0)

validation_set = BraTSDataset(val_folders,transform=transform)
validation_loader = DataLoader(validation_set,
                               batch_size=4,
                               shuffle=False,
                               num_workers=0)

for images, labels in training_loader:
    print(images.shape)
    print(labels.shape)
    break

for images, labels in validation_loader:
    print(images.shape)
    print(labels.shape)
    break


torch.Size([4, 4, 1, 128, 128, 128])
torch.Size([4, 4, 128, 128, 128])
torch.Size([4, 4, 1, 128, 128, 128])
torch.Size([4, 4, 128, 128, 128])


In [14]:
import monai
from monai.networks.nets import UNet
import torch.optim as optim


class BrainTumorModel(nn.Module):
    def __init__(self, in_channels=4, out_channels=4):  # out_channels=1 olarak değiştirildi
        super(BrainTumorModel, self).__init__()
        
        self.model = UNet(
            spatial_dims=3,
            in_channels=in_channels,
            out_channels=out_channels,
            channels=(16, 32, 64, 128, 256),
            strides=(2, 2, 2, 2),
            num_res_units=2
        )

        for param in list(self.model.parameters())[:-10]:
            param.requires_grad = False

    def forward(self, x):
        return self.model(x)

In [15]:
def train_model(model,training_loader,validation_loader,num_epochs=50):
    device = torch.device('cpu')

    model.to(device)

    criterion = monai.losses.DiceLoss(sigmoid=True)
    optimizer = optim.Adam(model.parameters(), lr = 1e-4)

    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5)

    for epochs in range(num_epochs):

        model.train()
        train_loss = 0

        for images,labels in training_loader:
            inputs = images.squeeze(2).to(device)
            print(inputs.shape)

            labels = labels.to(device)
            print(labels.shape)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs,labels)
            loss.backward()
            optimizer.step()

            train_loss +=loss.item()

        model.eval()
        val_loss = 0

        with torch.no_grad():
            for images,labels in validation_loader:
                inputs = images.squeeze(2).to(device)
                print(inputs.shape)
                labels = labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

        scheduler.step(val_loss)

        print(f'Epoch {epochs+1}/{num_epochs}')
        print(f'Training Loss: {train_loss/len(training_loader):.4f}')
        print(f'Validation Loss: {val_loss/len(validation_loader):.4f}')


In [16]:
model = BrainTumorModel()

train_model(model,training_loader,validation_loader,50)

torch.Size([4, 4, 128, 128, 128])
torch.Size([4, 4, 128, 128, 128])
torch.Size([4, 4, 128, 128, 128])
torch.Size([4, 4, 128, 128, 128])
torch.Size([4, 4, 128, 128, 128])
torch.Size([4, 4, 128, 128, 128])
torch.Size([4, 4, 128, 128, 128])
torch.Size([4, 4, 128, 128, 128])
torch.Size([4, 4, 128, 128, 128])
torch.Size([4, 4, 128, 128, 128])
torch.Size([4, 4, 128, 128, 128])
torch.Size([4, 4, 128, 128, 128])
torch.Size([4, 4, 128, 128, 128])
torch.Size([4, 4, 128, 128, 128])
torch.Size([4, 4, 128, 128, 128])
torch.Size([4, 4, 128, 128, 128])
torch.Size([4, 4, 128, 128, 128])
torch.Size([4, 4, 128, 128, 128])
torch.Size([4, 4, 128, 128, 128])
torch.Size([4, 4, 128, 128, 128])
torch.Size([4, 4, 128, 128, 128])
torch.Size([4, 4, 128, 128, 128])
torch.Size([4, 4, 128, 128, 128])
torch.Size([4, 4, 128, 128, 128])
torch.Size([4, 4, 128, 128, 128])
torch.Size([4, 4, 128, 128, 128])
torch.Size([4, 4, 128, 128, 128])
torch.Size([4, 4, 128, 128, 128])
torch.Size([4, 4, 128, 128, 128])
torch.Size([4,

KeyboardInterrupt: 